R003
Jhilik Biswas
MBATech AI
B1

In [3]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
import numpy as np
from tensorflow.keras import layers, models, optimizers
import tensorflow.keras.backend as k
import matplotlib.pyplot as plt
import random

In [4]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

In [7]:
train_images = train_images.astype('float32')/255.0
test_images = test_images.astype('float32')/255.0  #(255 used to normalize)

In [8]:
def make_pairs(images, labels):
  """Generates Pairs of images from dataset"""
  num_classes = max(labels) + 1
  digit_indices = [np.where(labels == i)[0] for i in range(num_classes)]
 
  pairs, pair_labels = [], []
  for idx1 in range(len(images)):
    x1 = images[idx1]
    label = labels[idx1]
    idx2 = random.choice(digit_indices[label])
    x2 = images[idx2]
    pairs += [[x1, x2]]
    pair_labels += [1]
 
    label2 = (label + random.randrange(1, num_classes)) % num_classes
    idx2 = random.choice(digit_indices[label2])
    x2 = images[idx2]
    pairs += [[x1, x2]]
    pair_labels += [0]
 
  return np.array(pairs), np.array(pair_labels)

In [9]:
train_pairs, train_pair_labels = make_pairs(train_images, train_labels)
test_pairs, test_pair_labels = make_pairs(test_images, test_labels)

In [11]:
print("'first 10 pairs' labels:")
print(train_pair_labels[:10])

'first 10 pairs' labels:
[1 0 1 0 1 0 1 0 1 0]


In [20]:
def build_siamese_model(input_shape):
  """Builds a siamese Network Model"""
  input = layers.Input(shape= input_shape)
  x=layers.Conv2D(64, (3,3), activation='relu')(input)
  x=layers.MaxPooling2D(2,2)(x)
  x=layers.Conv2D(128, (3,3), activation='relu')(x)
  x=layers.MaxPooling2D(2,2)(x)
  x=layers.Flatten()(x)
  x=layers.Dense(128, activation='relu')(x)
  model=models.Model(input,x)
  return model

In [21]:
def build_siamese_network(input_shape):
  """Builds the full Siamese Network Architecture"""
  base_network = build_siamese_model(input_shape)
 
  input_a = layers.Input(shape=input_shape)
  input_b = layers.Input(shape=input_shape)
 
  processed_a = base_network(input_a)
  processed_b = base_network(input_b)
 
  distance = tf.keras.layers.Subtract()([processed_a, processed_b])
  #distance = tf.keras.layers.Lambda(lambda x: tf.abs(tensors[0] - tensors[1]))(distance)
 
  output = layers.Dense(1, activation="sigmoid")(distance)
 
  siamese_net = models.Model([input_a, input_b], output)
  return siamese_net

In [22]:
def train_siamese_network_optimized(siamese_network, train_pairs, train_pair_labels, test_pairs, test_pair_labels):
  train_subset = 10000
  test_subset = 2000
 
  siamese_network.fit(
      [train_pairs[:train_subset, 0], train_pairs[:train_subset, 1]], train_pair_labels[:train_subset],
      batch_size = 64, epochs = 5,
      validation_data = ([test_pairs[:test_subset, 0], test_pairs[:test_subset, 1]], test_pair_labels[:test_subset])
  )

In [24]:
def contrastive_loss(y_true, y_pred, margin=1.0):
    y_true = tf.cast(y_true, y_pred.dtype)
    square_pred = tf.square(y_pred)
    margin_square = tf.square(tf.maximum(margin - y_pred, 0))
    return tf.reduce_mean(
        y_true * square_pred + (1 - y_true) * margin_square
    )

In [25]:
input_shape = (28, 28, 1)
siamese_network = build_siamese_network(input_shape)
siamese_network.compile(loss=contrastive_loss, optimizer="adam", metrics=["accuracy"])

In [26]:
train_siamese_network_optimized(siamese_network, train_pairs, train_pair_labels, test_pairs, test_pair_labels)

Epoch 1/5
157/157 ━━━━━━━━━━━━━━━━━━━━ 10s 50ms/step - accuracy: 0.5049 - loss: 0.2500 - val_accuracy: 0.4445 - val_loss: 0.2500
Epoch 2/5
157/157 ━━━━━━━━━━━━━━━━━━━━ 7s 46ms/step - accuracy: 0.4937 - loss: 0.2500 - val_accuracy: 0.5195 - val_loss: 0.2500
Epoch 3/5
157/157 ━━━━━━━━━━━━━━━━━━━━ 7s 46ms/step - accuracy: 0.5192 - loss: 0.2500 - val_accuracy: 0.4980 - val_loss: 0.2499
Epoch 4/5
157/157 ━━━━━━━━━━━━━━━━━━━━ 7s 46ms/step - accuracy: 0.5000 - loss: 0.2500 - val_accuracy: 0.5040 - val_loss: 0.2499
Epoch 5/5
157/157 ━━━━━━━━━━━━━━━━━━━━ 7s 47ms/step - accuracy: 0.4835 - loss: 0.2499 - val_accuracy: 0.5145 - val_loss: 0.2500
